In [ ]:
import os
import re
import io
import sys
import glob
import json
import dask
import requests
import datetime
import urllib
import psycopg2
import numpy as np
import pandas as pd

import dask.diagnostics
import sqlalchemy as db
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
%aimport opencell.database.operations
%aimport opencell.database.uniprot_utils
from opencell.database import models, operations, ms_utils, uniprot_utils
from opencell.database import utils as db_utils

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-test.json')
url = db_utils.url_from_credentials('../../db-credentials-dev.json')
url = db_utils.url_from_credentials('../../db-credentials-cap.json')

engine = db.create_engine(url)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)
url

In [ ]:
d = pd.read_sql(
    '''
    select pml_id, date, count(*) as num_fovs from microscopy_fov
    left join microscopy_dataset using (pml_id)
    group by pml_id, date
    order by pml_id
    ''',
    engine
)

In [ ]:
d['date'] = pd.to_datetime(d.date)

In [ ]:
d.index = d.date
d = d.sort_index()

In [ ]:
plt.plot(d.index, d.num_fovs)

In [ ]:
plt.plot(d.index, d.num_fovs.rolling('7d').sum())